# Imports

In [1]:
%reload_ext autoreload
%autoreload 2


import warnings
warnings.filterwarnings("ignore")

import requests


import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import formataddr

import pandas as pd
import time, requests, csv, json
from datetime import datetime
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 20000)
pd.set_option('display.max_colwidth', 10000)

import sys, os, shutil, pdb
sys.path.insert(1, '/Users/muhammadfaisal/Documents/')

from pathlib import Path
import urllib.request

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, clear_output
from bs4 import BeautifulSoup


from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

try:
  from tabula.io import read_pdf
except:
  %pip install tabula-py
  
try: # running remotely
    from google.colab import drive 
except: # running locally
    from utils import *
    from engine import GDrive
    

In [2]:
TEST_CATALOGS=[
    # 'Medcare_BestSelling',
    # 'Medcare_CLOSEOUT',
    # 'Medcare_MASTER'
]

TEST_CATALOGS=[x.lower() for x in TEST_CATALOGS]

# Local VS Remote

In [3]:
def download_scripts():
    # downloading utils and engine scripts from github
    r=requests.get("https://raw.githubusercontent.com/faysal887/tricyons-analyser-v2/master/engine.py")
    with open('engine.py', 'w') as f: f.write(r.content.decode("utf-8"))

    r=requests.get("https://raw.githubusercontent.com/faysal887/tricyons-analyser-v2/master/utils.py")
    with open('utils.py', 'w') as f: f.write(r.content.decode("utf-8"))

In [4]:
'''
checking whether the code is running on local machine or on collab
'''

LOCAL=None # running code locally or remotely
try:
    drive.mount('/content/drive')

    GoogleAuth.DEFAULT_SETTINGS['client_config_file'] = '/content/drive/MyDrive/Colab Notebooks/client_secrets.json'
    gauth = GoogleAuth()
    gauth.GetAuthUrl()
    print('running on collab')
    LOCAL=False
except Exception as e:
    print(e)
    print('running locally')
    LOCAL=True


if LOCAL: # running locally
    try:
        if g: print('GDrive already connected!')
    except: 
        g=GDrive()
        print('created new connection!')
else: # running remotely
    drive.mount('/content/drive')

    GoogleAuth.DEFAULT_SETTINGS['client_config_file'] = '/content/drive/MyDrive/Colab Notebooks/client_secrets.json'
    gauth = GoogleAuth()
    print(gauth.GetAuthUrl())
    redirect_url=input('Enter redirect url')
    code=redirect_url.split('code=')[1].split('&scope=')[0]
    gauth.Authenticate(code)
    download_scripts()
    from utils import *
    from engine import GDrive
    g=GDrive(gauth)



name 'drive' is not defined
running locally
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=454591216913-28jl16cqrsojhifrtmopf0ts9mb4281t.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
created new connection!


# Download Catalogs

In [5]:
TMP_DIR='./tmp'
try: shutil.rmtree(TMP_DIR)
except: pass

In [6]:
# main

tmp_dir='./tmp'
merged_dir=f'{tmp_dir}/merged'
online_catalogs=f'{tmp_dir}/online_catalogs'
gdrive_catalogs=f'{tmp_dir}/gdrive_catalogs'

# 1. create /tmp if not
Path(tmp_dir).mkdir(parents=True, exist_ok=True)
Path(merged_dir).mkdir(parents=True, exist_ok=True)
Path(online_catalogs).mkdir(parents=True, exist_ok=True)
Path(gdrive_catalogs).mkdir(parents=True, exist_ok=True)
print('done 1')

# 2. download excel sheet for links and emails 
tree=['Supplier Catalogues', 'AUTOMATION', 'Brands For Auto-Email']
folder_id=g.find_gdrive_folder_id(tree=tree, folder_id='root')
g.download_files(folder_id, tmp_dir)

linksdf=pd.read_excel(f'{tmp_dir}/Catalogs Link.xlsx')
# data=linksdf.dropna(subset=['Link'], axis=0)
data=strip_column_names(linksdf)
print(data.shape)
print('done 2')



done 1
(41, 16)
done 2


In [7]:
# 3. download excel catalogs
df=data[(data.Active==True) & (data.Type=='LINK')]
print(df.shape)
error_catalogs, catalogs=download_online_excel_catalogs(df, online_catalogs, test_catalogs=TEST_CATALOGS)
print('done 3')

catalogs.keys()

(24, 16)
ERROR:  drachmatrading 'URL LINK' <class 'KeyError'> utils.py 319
(82, 4)
ERROR:  ecomwholesaledeals 'DataFrame' object has no attribute 'append' <class 'AttributeError'> utils.py 238
ERROR:  minmaxdeals single positional indexer is out-of-bounds <class 'IndexError'> utils.py 249
ERROR:  shepher_california_available Couldn't find a tree builder with the features you requested: xml. Do you need to install a parser library? <class 'bs4.FeatureNotFound'> utils.py 307
ERROR:  shepher_california_special Couldn't find a tree builder with the features you requested: xml. Do you need to install a parser library? <class 'bs4.FeatureNotFound'> utils.py 307
ERROR:  shepher_newjersey_available Couldn't find a tree builder with the features you requested: xml. Do you need to install a parser library? <class 'bs4.FeatureNotFound'> utils.py 307
ERROR:  shepher_newjersey_special Couldn't find a tree builder with the features you requested: xml. Do you need to install a parser library? <class 

dict_keys(['BuyWholesaleToday', 'DallasWholesaleCompany', 'EmpireDistribution', 'EpilsonWholesale', 'GSCommodityTrading', 'KNTradingLLC', 'KoleImports', 'LvDistribution', 'Medcare_BestSelling', 'Medcare_CLOSEOUT', 'Medcare_MASTER', 'NexDeal_1', 'NexDeal_2', 'PalletFly', 'WeinersLtd'])

In [8]:
error_catalogs

{'drachmatrading': KeyError('URL LINK'),
 'ecomwholesaledeals': AttributeError("'DataFrame' object has no attribute 'append'"),
 'minmaxdeals': IndexError('single positional indexer is out-of-bounds'),
 'shepher_california_available': bs4.FeatureNotFound("Couldn't find a tree builder with the features you requested: xml. Do you need to install a parser library?"),
 'shepher_california_special': bs4.FeatureNotFound("Couldn't find a tree builder with the features you requested: xml. Do you need to install a parser library?"),
 'shepher_newjersey_available': bs4.FeatureNotFound("Couldn't find a tree builder with the features you requested: xml. Do you need to install a parser library?"),
 'shepher_newjersey_special': bs4.FeatureNotFound("Couldn't find a tree builder with the features you requested: xml. Do you need to install a parser library?"),
 'shepher_newyork_special': bs4.FeatureNotFound("Couldn't find a tree builder with the features you requested: xml. Do you need to install a par

In [9]:
# 4. delete existing supplier folders
tree_base=['Supplier Catalogues', 'AUTOMATION', 'Catalogs_ALL']
df.Distributor.apply(lambda x: g.delete_by_name(tree_base+[x]))
print('done 4')

done 4


In [10]:
# 5. create new supplier folders
tree=['Supplier Catalogues', 'AUTOMATION', 'Catalogs_ALL']
parent_folder_id=g.find_gdrive_folder_id(tree=tree, folder_id='root')
df['folder_id'] = df.Distributor.apply(lambda x: g.create_folder(parent_folder_id, x))
print('done 5')

# 6. create folder for catalog in supplier folder
df['catalog_folder_id'] = df.folder_id.apply(lambda x: g.create_folder(parent_folder_id=x, folder_name='catalog'))
print('done 6')



done 5
done 6


In [11]:
# 7. upload catalog
# _ = df.apply(lambda x: g.upload_file(folder_id=x.catalog_folder_id, file_path=f'{online_catalogs}/{x.Distributor}.xlsx'), axis=1)
for i, row in df.iterrows():
    print(f'{i+1}/{len(df)}', end='\r')
    file_path=f'{online_catalogs}/{row.Distributor}.xlsx'
    if os.path.exists(file_path):
        g.upload_file(folder_id=row.catalog_folder_id, file_path=file_path)
    
print('done 7')


# 8. delete /tmp
# shutil.rmtree(tmp_dir)
# print('done 8')

done 7


In [12]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
                                # PART-2: Start again with data #
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
linksdf=pd.read_excel(f'{tmp_dir}/Catalogs Link.xlsx')
# data=linksdf.dropna(subset=['Link'], axis=0)
data=strip_column_names(linksdf)
if TEST_CATALOGS:
    df=data[(data.Active==True)&(data.Distributor.str.lower().isin(TEST_CATALOGS))].copy()
else:
    df=data[(data.Active==True)].copy()
    
print(df.shape)

(31, 16)


In [13]:
# 8. for all the suppliers in catalogs_all, get their catalog folder_ids 
for i, row in df.iterrows():
    try:
        print(f'{i+1}/{len(df)}', end='\r')
        tree=['Supplier Catalogues', 'AUTOMATION', 'Catalogs_ALL', row.Distributor, 'catalog']
        catalog_folder_id=g.find_gdrive_folder_id(tree=tree, folder_id='root')
        df.loc[i, 'catalog_folder_id']=catalog_folder_id
    except Exception as e:
        print('Error: ', row['Distributor'], e)
print('done 8')

done 8


In [14]:
# 9. loop on supplier folders id df and download all catalogs from gdrive
# _ = df.catalog_folder_id.apply(lambda x: g.download_files(gdrive_folder_id=x, local_folder_path=gdrive_catalogs))

for i, row in df[['Distributor','catalog_folder_id']].iterrows():
    print(f'{i+1}/{len(df)}', end='\r')

    try:
        dist, _id=row['Distributor'], row['catalog_folder_id']
        g.download_files(gdrive_folder_id=_id, local_folder_path=gdrive_catalogs)
        time.sleep(3)
    except Exception as e:
        print('Error: ', dist, e)
print('done 9')

done 9


In [15]:
# 10. create new catalogs dict obj

catalogs={}
for i, fn in enumerate(os.listdir(gdrive_catalogs)):
    print(f'{i+1}/{len(os.listdir(gdrive_catalogs))}', end='\r')

    if '.xlsx' in fn or 'xls' in fn: tmpdf=pd.read_excel(f'{gdrive_catalogs}/{fn}')
    if '.csv'   in fn: tmpdf=pd.read_csv(f'{gdrive_catalogs}/{fn}')

    fn=fn.split('.')[0]

    catalogs[fn]=tmpdf
    # catalogs[fn.replace('.xlsx','').replace('.csv','').replace('.csv','')]=tmpdf

print('done 10')


done 10


In [16]:
# catalogs['MinMaxDeals']=catalogs['MinMaxDeals'].iloc[4:,]
# catalogs['MinMaxDeals']=make_first_row_header(catalogs['MinMaxDeals'])
# catalogs['MinMaxDeals']=strip_column_names(catalogs['MinMaxDeals'])
# catalogs['LvDistribution'].head()

# LvDistribution, MinMaxDeals

In [17]:
# df.loc[df.Distributor=='SU_Catalogs_Merged', 'Active']=False
# df=df.drop([39])
# df

# df=df[df.Distributor.isin(['WeinersLtd', 'SU_Catalogs_Merged', 'DPCIWholesale'])]

In [18]:
# sanity checks to check all the required columns are present
for name, tmpdf in catalogs.items():
    try:
        # if name!='DPCI_Wholesale': continue
        # pdb.set_trace()
        id_columns_name	= df[df.Distributor==name].id_columns_name.item()
        id_columns_type = df[df.Distributor==name].id_columns_type.item()
        price_column    = df[df.Distributor==name].price_column.item()

        if id_columns_name not in tmpdf.columns: print(f'{name} "{id_columns_name}" column missing')
        if price_column not in tmpdf.columns: print(f'{name} "{price_column}" column missing')
    except Exception as e:
        print(f'Supplier: {name}')
        print(f'ERROR: {e}')
        print(f'id_columns_name: {id_columns_name}')
        print(f'id_columns_type: {id_columns_type}')
        print(f'price_column: {price_column}')


        # print(price_column, id_columns_name, id_columns_type)
        # print('\n')


Supplier: DPCIWholesale
ERROR: can only convert an array of size 1 to a Python scalar
id_columns_name: UPC
id_columns_type: upc
price_column: price


In [24]:
# 11. label data
df, catalogs=label_data(df, catalogs)
print('done 11')

All Suppliers are Labeled
Error in DPCIWholesale
can only convert an array of size 1 to a Python scalar <class 'ValueError'> utils.py 161
done 11


In [25]:
# preprocess each supplier separately
for name, tmpdf in catalogs.items():
    if name == 'NUK':                tmpdf=preprocess_nuk(tmpdf)
    if name == 'BGSales':            tmpdf=preprocess_bgsales(tmpdf)
    if name == 'EcomWholesaleDeals': tmpdf=preprocess_ewd(tmpdf)

    catalogs[name]=tmpdf

In [28]:
# label encode supplier name in df dataframe
# keys=df.Distributor.unique().tolist()
# values=['TRI'+str(i).zfill(5) for i in range(len(keys))]
# labels_map=dict(zip(keys, values))
# df['distributor_key'] = df.Distributor.map(labels_map)

# now this is being done manually
# print('done 12')


# apply labels_map for each catalog in catalog dict of dfs
for key, tmpdf in catalogs.items():
    try:
        tmpdf['distributor_key']=df[df.Distributor==key].distributor_key.item()
        catalogs[key]=tmpdf
    except Exception as e:
        print(f'Error in {key}')
        print(e)

# 12. upload updated catalog_links df
# df.drop(['folder_id', 'catalog_folder_id'], axis=1).to_excel(f'{tmp_dir}/Catalogs Link.xlsx', index=False)

Error in DPCIWholesale
can only convert an array of size 1 to a Python scalar


In [29]:
# 13. check for missing values in each column
missing_total=0
for name, tmpdf in catalogs.items():
    try:
        # print(name, tmpdf['each_price'].isnull().sum())
        missing_total+=tmpdf['each_price'].isnull().sum()
    except Exception as e:
        print(f'{name}: {e}')
print(missing_total)
print('done 12')

DPCIWholesale: 'each_price'
59
done 12


In [30]:
# 13. preprocess asins
asins_suppliers=df[df.id_columns_type=='asin'].Distributor.tolist()

tmpli=[]
for x in asins_suppliers:
  try:
    print(x, catalogs[x].shape)
    tmpli.append(catalogs[x][['asin', 'each_price', 'distributor_key']])
  except Exception as e:
    print(f'Error in {x}: ', e)

if tmpli:    
  asinsdf = pd.concat(tmpli)
  print('after concating all asinsdf: ',asinsdf.shape, asinsdf.each_price.isnull().sum())

  asinsdf=asinsdf.dropna()
  print('after dropna: ',asinsdf.shape, asinsdf.each_price.isnull().sum())

  asinsdf['each_price']=asinsdf.each_price.apply(pd.to_numeric, errors="coerce")
  asinsdf=asinsdf.dropna(subset=['each_price'])
  print('after price column check: ',asinsdf.shape)

  asinsdf=preprocess_price(asinsdf)
  print('after preprocess_price: ',asinsdf.shape, asinsdf.each_price.isnull().sum())

  asinsdf=asinsdf.reset_index(drop=True)
  print('after reset_index: ',asinsdf.shape, asinsdf.each_price.isnull().sum())

  asinsdf.to_excel(f'{merged_dir}/asinsdf.xlsx', index=False)
else:
  asinsdf=pd.DataFrame()

print('after saving: ', asinsdf.shape)
print('done 13')

BuyWholesaleToday (126, 8)
DallasWholesaleCompany (1026, 10)
Error in DrachmaTrading:  'DrachmaTrading'
EmpireDistribution (28, 10)
EpilsonWholesale (17, 11)
GSCommodityTrading (208, 12)
KNTradingLLC (60, 10)
LvDistribution (107, 14)
Error in MinMaxDeals:  'MinMaxDeals'
NexDeal_1 (370, 19)
NexDeal_2 (208, 16)
PalletFly (9638, 28)
WholesaleFortune (31, 11)
after concating all asinsdf:  (11819, 3) 36
after dropna:  (11740, 3) 0
after price column check:  (10284, 3)
after preprocess_price:  (10284, 3) 0
after reset_index:  (10284, 3) 0
after saving:  (10284, 3)
done 13


In [31]:
# 14. preprocess upcs
upc_suppliers=df[df.id_columns_type=='upc'].Distributor.tolist()
# upcsdf = pd.concat([catalogs[x][['upc', 'each_price', 'distributor_key']] for x in upc_suppliers])
tmpli=[]
for x in upc_suppliers:
  try:
    tmpli.append(catalogs[x][['upc', 'each_price', 'distributor_key']])
  except Exception as e:
    print(f'Error in {x}: {e}')

if tmpli:
  upcsdf = pd.concat(tmpli)
  print('after concat: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

  upcsdf=upcsdf.dropna(subset=['upc', 'each_price'])
  print('after dropna: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

  upcsdf=preprocess_upc(upcsdf)
  print('after preprocess_upc: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

  upcsdf=preprocess_price(upcsdf)
  print('after preprocess_price: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

  upcsdf=upcsdf.reset_index(drop=True)
  print('after reset_index: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

  upcsdf.to_excel(f'{merged_dir}/upcsdf.xlsx', index=False)
else: 
  upcsdf=pd.DataFrame()
  
print('after save: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

print('done 14')

Error in DPCI_Wholesale: 'DPCI_Wholesale'
Error in EcomWholesaleDeals: 'EcomWholesaleDeals'
Error in Shepher_California_Available: 'Shepher_California_Available'
Error in Shepher_California_Special: 'Shepher_California_Special'
Error in Shepher_NewJersey_Available: 'Shepher_NewJersey_Available'
Error in Shepher_NewJersey_Special: 'Shepher_NewJersey_Special'
Error in Shepher_NewYork_Special: 'Shepher_NewYork_Special'
Error in Shepher_NewYork_Website: 'Shepher_NewYork_Website'
after concat:  (110991, 3) 23
after dropna:  (109166, 3) 0
after preprocess_upc:  (108918, 3) 0
after preprocess_price:  (108918, 3) 0
after reset_index:  (108918, 3) 0
after save:  (108918, 3) 0
done 14


In [32]:
# 15. upload merged asin and upc on gdrive
tree=['Supplier Catalogues', 'AUTOMATION', 'ScanUnlimited_ALL']
folder_id=g.find_gdrive_folder_id(tree, folder_id='root')

# delete existing ScanUnlimited_ALL
g.delete_by_id(folder_id)

# create new ScanUnlimited_ALL
tree=['Supplier Catalogues', 'AUTOMATION']
parent_folder_id=g.find_gdrive_folder_id(tree, folder_id='root')
new_folder_id=g.create_folder(parent_folder_id, 'ScanUnlimited_ALL')

if not asinsdf.empty: g.upload_file(new_folder_id, f'{merged_dir}/asinsdf.xlsx')
if not upcsdf.empty: g.upload_file(new_folder_id, f'{merged_dir}/upcsdf.xlsx')
print('done 16')

done 16


In [33]:
print(f'''
    Total:   {asinsdf.shape[0]+upcsdf.shape[0]}
    upcsdf:  {upcsdf.shape[0]}
    asinsdf: {asinsdf.shape[0]}
''')


    Total:   119202
    upcsdf:  108918
    asinsdf: 10284



In [34]:
try: shutil.rmtree(TMP_DIR)
except: pass

In [35]:
asinsdf.distributor_key.value_counts()

distributor_key
TRI00009    9599
TRI00007     369
TRI00008     208
TRI00036      60
TRI00024      31
TRI00033      17
Name: count, dtype: int64

In [36]:
upcsdf.distributor_key.value_counts()

distributor_key
TRI00006    98425
TRI00031     4686
TRI00040     1578
TRI00030     1309
TRI00020     1214
TRI00029     1124
TRI00039      275
TRI00037      188
TRI00042      119
Name: count, dtype: int64

# TODO

In [ ]:
# add sheet name column so it shortens the download function
# change ignore to active
# add other columns list column

# Send Emails

In [ ]:
def send_email(addr, subject, body):
    try:
        fromaddr = "contact@tricyons.com"
        pswd     = "tricyons123"

        msg = MIMEMultipart()
        msg['From'] = formataddr(('Tricyons LLC', fromaddr))
        msg['To'] = addr
        msg['Subject'] = subject
        body = body

        msg.attach(MIMEText(body, 'plain'))
        # smtpserver = smtplib.SMTP("mail.tricyons.com", 465) # tls

        server = smtplib.SMTP("mail.tricyons.com", 26) # non-tls
        server.starttls()
        server.login(fromaddr, pswd)

        text = msg.as_string()
        server.sendmail(fromaddr, addr, text)
        server.quit()    
        return True
    except Exception as e:
        print('*****************************************')
        print(addr)
        print(e)
        print('*****************************************\n\n')
        return False


### Ask Catalog

In [ ]:
emailsdf=data[(data.Active==True) & (data.Type=='EMAIL')].sort_values('Email', ascending=False)
print(emailsdf.shape)
emailsdf

In [ ]:
with open('./email_catalog.txt', 'r') as f:
    email_body_template = f.read()

print(email_body_template)

In [ ]:
email_subject='Updated Catalog'

for i, row in emailsdf.iterrows():
    print(f'{i+1}/{emailsdf.shape[0]}', end='\r')

    email_to=row.Email.strip().lower()

    email_body=email_body_template.replace('SUBJECT_NAME_TAG', row.Email_Subject)

    email_sent = send_email(email_to, email_subject, email_body)
    if not email_sent: print(f'Error in sending email to: {row.Email_Subject}')

    time.sleep(3)

### New Suppliers

In [ ]:
data=pd.read_excel('/Users/muhammadfaisal/Downloads/SmartScout - Suppliers.xlsx')
print(data.shape)

data.head(2)

In [ ]:
df=data[~data.Email.isnull()]
df.shape

In [ ]:
with open('./email_template_new_account.txt', 'r') as f:
    email_body_template = f.read()

print(email_body_template)

In [ ]:
email_subject='Wholesale Account Opening'

for i, row in df.iterrows():
    print(f'{i+1}/{df.shape[0]}', end='\r')

    email_to=row.Email.strip().lower()

    email_body=email_body_template.replace('SUPPLIER_NAME_TAG', row.Name)

    email_sent = send_email(email_to, email_subject, email_body)

    if not email_sent: print(f'Error in sending email to: {email_to}')

    time.sleep(3)

### Account Open

In [ ]:


with open('./email_template.txt', 'r') as f:
    email_body_template = f.read()

print(email_body_template)


# email_to='faysalaslam887@gmail.com'
# email_subject = "Testing Email Body 3 Spaces"
# email_body=email_body_template.replace('COMPANY_NAME_TAG', 'Test Company')
# send_email(email_to, email_subject, email_body)

# g=GDrive()


'''
tmp_dir='./tmp/'

tree=['Supplier Catalogues', 'Brands For Auto-Email']
g.supplier_id=g.find_gdrive_folder_id(tree=tree, folder_id='root')
g.download_files(g.supplier_id, tmp_dir)

all_suppliers=pd.read_excel(f'{tmp_dir}/All Suppliers.xlsx')
print(all_suppliers.shape)

all_suppliers.head()

df=all_suppliers.dropna(subset=['Email'], axis=0).reset_index(drop=True)
print(df.shape)
df.head()

# email_subject = "Wholesale Account Opening"

# for i, row in df.iloc[78:].iterrows():
#     print(f'{i+1}/{df.shape[0]}', end='\r')

#     email_to=row.Email.strip().lower()

#     supplier_name = row.Name.strip()

#     email_body=email_body_template.replace('COMPANY_NAME_TAG', supplier_name)

#     email_sent = send_email(email_to, email_subject, email_body)
#     if not email_sent: print(f'Error in sending email to: {supplier_name}')

#     time.sleep(3)
    
'''

# Test New Catalogs

In [ ]:
from utils import *

In [ ]:
url=close

In [ ]:
import requests, json

import urllib.request
response = urllib.request.urlopen(best)
html = response.read()

In [ ]:
import urllib.request
urllib.request.urlretrieve(url, 'test.xlsx')

In [ ]:
df1=get_catalog_google_sheets(url).reset_index(drop=True)
print(df1.shape)
df1.head()


In [ ]:
df2=get_catalog_google_sheets_2(url, sheet_name='Has UPC')
print(df2.shape)
df2.head()

In [ ]:
df3=get_catalog_google_sheets_3(url)
df3.shape


In [ ]:
df4=get_catalog_google_sheets_4(url)
print(df4.shape)
df4.head()

In [ ]:
# EpilsonWholesale
df2=get_catalog_google_sheets_2(best, sheet_name='Wholesale Best-Selling List')
# df2=df2.iloc[1:]
# df2=make_first_row_header(df2)
# df2=df2[~df2.ASIN.astype(str).str.contains('--')]
# df2=df2.dropna()

print(df2.shape)
df2.head()

In [ ]:
# GSCommodityTrading
df1=get_catalog_google_sheets(url).reset_index(drop=True)
print(df1.shape)
df1=df1.iloc[9:]
df1.head(20)


In [ ]:
# MinMaxDeals
df1=get_catalog_google_sheets(url).reset_index(drop=True)
print(df1.shape)

In [ ]:
# KNTradingLLC
df2=get_catalog_google_sheets_2(url, sheet_name='Sheet1')
df2=make_first_row_header(df2)
df2=df2[[x for x in df2.columns.tolist() if x!=None]]


In [ ]:
# EcomWholesaleDeals 



In [ ]:
links={
    'EpilsonWholesale': "https://img1.wsimg.com/blobby/go/271277f5-5f94-472c-b153-578386146f02/downloads/COSMETIC%20SPREED%20SHEET%20%20(1).xlsx?ver=1680857754932",
    'GSCommodityTrading': "https://docs.google.com/spreadsheets/d/1EJ8kbB89SYv5fYeeCgds5ue7p_3nyckyN_yujUquJeo/edit#gid=0",
    'MinMaxDeals': "https://docs.google.com/spreadsheets/d/151_G7FTLfJP7y9kypBEF4sYN7gICIrYT/edit",
    'KNTradingLLC': "https://06315f73-014b-478e-b312-e53df7038d3a.filesusr.com/ugd/cc4fa6_9d8a073014d4499d8d117b6724916ea8.xlsx?dn=3_23%20KN%20INVENTORY.xlsx",
    'EcomWholesaleDeals': '1EIzkO1ES0v1VcabxOUQ7VaREIjMnxSgB'
}


In [ ]:
url=links['GSCommodityTrading']
url

In [ ]:
df1=get_catalog_google_sheets(url).reset_index(drop=True)
print(df1.shape)
df1=df1.iloc[9:]
df1.head()


In [ ]:
df2=get_catalog_google_sheets_2(url, sheet_name='Sheet1')
df2=make_first_row_header(df2)

df2=df2[[x for x in df2.columns.tolist() if x!=None]]
# df2=df2.dropna()

print(df2.shape)
df2.head()

In [ ]:
df3=get_catalog_google_sheets_3(url)
df3.shape


In [ ]:
df4=get_catalog_google_sheets_4(url)
print(df4.shape)
df4.head()

In [ ]:
[ x.lower() for x in ['Medcare_BestSelling', 'Medcare_MASTER', 'Medcare_CLOSEOUT']]

In [ ]:
# distributor_key
# TRI00006    98543
# TRI00029     1127
# TRI00042      120

In [ ]:
best = "https://docs.google.com/spreadsheets/d/1dB6cI-7zBOBjll1XTV1Yav1nP_3xPEPdfK0TTZo_yHA/export?format=xlsx&gid=1640087997"

best=get_catalog_google_sheets_2(best, sheet_name='Wholesale Best-Selling List')

best=best.iloc[1:,]

best=make_first_row_header(best)
best=strip_column_names(best)

print(best.shape)
best.head()

In [ ]:
master='https://docs.google.com/spreadsheets/d/1CGv5t5euc-OH64MCpiVGpwYC9ObCm5ZrceVgm_88o8I/export?format=xlsx&gid=1450292243'

master=get_catalog_google_sheets_2(master, sheet_name='Wholesale Master List')

master=master.iloc[1:,]

master=make_first_row_header(master)
master=strip_column_names(master)

print(master.shape)
master.head()


In [ ]:
close='https://docs.google.com/spreadsheets/d/15493Wi1IdPYtX8oYfsQszOTAUkBkMQYDy31wNI3bsoY/export?format=xlsx&gid=1221906338'

close=get_catalog_google_sheets_2(close, sheet_name='Has UPC')
close=close.iloc[1:,]

close=make_first_row_header(close)
close=strip_column_names(close)

print(close.shape)
close.head()

In [ ]:
close.shape[0]-close.UPC.isnull().sum()